In [1]:
import os
import pandas as pd
import requests
from getpass import getpass
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import time
from datetime import datetime

In [2]:
key = getpass()

········


In [3]:
class TFTRiotAPI:
    def __init__(self):
        pass
    
    def summoner_name(self, key, name):
        name = name.replace(' ', '%20')
        request = requests.get(f'https://br1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{name}?api_key={key}')
        return eval(request.content)
    
    def match_list(self, key, puuid, n = 30):
        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={n}&api_key={key}')
        return eval(request.content)
    
    def match_info(self, key, matchid, unixtime = False):
        '''Return a dataframe with some infos from the match (matchid, datetime, game lenght(s), game variation,
        game version, number of the set).

        key = riot API-key
        matchid = The id from a specific match of team fight tatics(TFT)'''

        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
        df = pd.DataFrame(pd.json_normalize(eval(request.content)))
        df2 = df[['metadata.match_id', 'info.game_datetime', 'info.game_length', 'info.game_variation',
           'info.game_version', 'info.tft_set_number']]
        df2.columns = [i.replace('metadata.', '').replace('info.', '') for i in df2.columns]
        
        y = df2.copy()
        if unixtime == False:
            x = df2['game_datetime'].apply(lambda x: pd.to_datetime(datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S')))
            y = df2.copy()
            y['game_datetime'] = x

        return y
    
    def match_participants(self, key, matchid):
        '''Return a list with the puuid from the 8 participants of the match'''

        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
        df = pd.DataFrame(pd.json_normalize(eval(request.content)))
        return df.iloc[0,2]
    
    def match_results(self, key, matchid):
        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
        soup = BeautifulSoup(request.content)
        df_master = pd.DataFrame(columns = ['match_id', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid',
                                     'time_eliminated', 'total_damage_to_players', 'Set3_Blademaster', 'Blaster',
                                     'Set3_Brawler', 'Set3_Celestial', 'Chrono', 'Cybernetic', 'DarkStar',
                                     'Demolitionist', 'Infiltrator', 'ManaReaver', 'MechPilot',
                                     'Mercenary', 'Set3_Mystic', 'Protector', 'Rebel', 'Sniper',
                                     'SpacePirate', 'StarGuardian', 'Starship', 'Set3_Sorcerer', 
                                     'Valkyrie', 'Vanguard', 'Set3_Void', 'champion_1', 'champion_2', 'champion_3',
                                     'champion_4', 'champion_5', 'champion_6', 'champion_7', 'champion_8', 'champion_9',
                                     'champion_10'])

        df_match = pd.DataFrame(pd.json_normalize(eval(soup.text)))
        dict_players = df_match.iloc[0,-3]

        for i in range(8):
            info_player = pd.DataFrame([dict_players[i]])
            df_master.loc[i, 'match_id'] = df_match.loc[0, 'metadata.match_id']
            df_master.loc[i, 'gold_left'] = info_player.loc[0, 'gold_left']
            df_master.loc[i, 'last_round'] = info_player.loc[0, 'last_round']
            df_master.loc[i, 'level'] = info_player.loc[0, 'level']
            df_master.loc[i, 'placement'] = info_player.loc[0, 'placement']
            df_master.loc[i, 'players_eliminated'] = info_player.loc[0, 'players_eliminated']
            df_master.loc[i, 'puuid'] = info_player.loc[0, 'puuid']
            df_master.loc[i, 'time_eliminated'] = info_player.loc[0, 'time_eliminated']
            df_master.loc[i, 'total_damage_to_players'] = info_player.loc[0, 'total_damage_to_players']

            for j in range(len(info_player.iloc[0, -2])): ## nesta etapa ele adiciona os tipos e quantidade, possui um valor extra no dicio que causa um bug (TemplateTrait)
                intermediate = pd.DataFrame(info_player.iloc[0, -2])
                df_master.loc[i, intermediate['name'][j]] = intermediate['num_units'][j]

            for k in range(len(info_player.iloc[0, -1])):
                intermediate = pd.DataFrame(info_player.iloc[0, -1])        
                df_master.loc[i, 'champion_' + str(k+1)] = (intermediate['character_id'][k] + '_' + str(intermediate['tier'][k])).replace('TFT3_', '')

        #df_master.columns = ['match_id', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid',
        #                             'time_eliminated', 'total_damage_to_players', 'blademaster_num', 'blaster_num',
        #                             'brawler_num', 'celestial_num', 'chrono_num', 'cybernetic_num', 'dark_star_num',
        #                             'demolitionist_num', 'infiltrator_num', 'mana_reaver_num', 'mech_pilot_num',
        #                             'mercenary_num', 'mystic_num', 'protector_num', 'rebel_num', 'sniper_num',
        #                             'space_pirate_num', 'star_guardian_num', 'starship_num', 'sorcerer_num', 
        #                             'valkyrie_num', 'vanguard_num', 'void_num', 'champion_1', 'champion_2', 'champion_3',
        #                             'champion_4', 'champion_5', 'champion_6', 'champion_7', 'champion_8', 'champion_9',
        #                             'champion_10']
        return df_master

In [4]:
riot = TFTRiotAPI()

# Adicionando novos puuids

In [5]:
matchids_atual = pd.read_csv('matchids.csv')['matchid']
matchids_atual

0       BR1_1929504335
1       BR1_1917107281
2       BR1_1925813673
3       BR1_1929034883
4       BR1_1927766146
             ...      
6194    BR1_1933014711
6195    BR1_1932950938
6196    BR1_1932935031
6197    BR1_1946164876
6198    BR1_1933140534
Name: matchid, Length: 6199, dtype: object

In [6]:
puuids_new = pd.read_csv('puuids.csv')
puuids_new

,puuid
0,M38b7eC1_Qp2WSJgAi9F3Sd8N07mSMoNxml_TLcjBVRP5M...
1,wpKYtw4TqiWFEqkf5bTEb3GPb4ts3cO2Kqw2t4E_7-yOya...
2,Ql2lf25SCGPkTx6FB1mYNP0YkfjnIQYpT8y4fMPwnaYtAY...
3,XbTawWZCXlhyVxw-1GGk77wOIJHMnBx7yPl6fqAl02Yd73...
4,ioMFXi24_6KA-0VOmhnkEYPIpLKufqo3wKGpsEqSHNle88...
...,...
4908,98zIDX2yvpOK1ucfc1HWvB_DO5ZmNLMX0gdSYkun_qJHWP...
4909,Xr17S4_7HSgWvyyLtTd0OaWwv_DEQmfB_9okWjyRtaDyLc...
4910,OOdIosU1cLGV5o57fEWxZnos5lmYC-Oh02DEa2cj1OBghN...
4911,Wb3hQuUoE1He97iCrZlUZB2SEdDmC__lDCbLrGa6F3PWnq...


In [13]:
for i in tqdm(matchids_atual):
    try:
        intermediate = riot.match_participants(key = key, matchid = i)
        for j in intermediate:
            puuids_new = puuids_new.append({'puuid' : j}, ignore_index = True)
    except:
        print('Error in matchid:', i)
        
    puuids_new.drop_duplicates().to_csv('puuids.csv', index = False)
    time.sleep(1)

 14%|█▍        | 885/6199 [39:40<3:54:03,  2.64s/it]

Error in matchid: BR1_1926540722


 34%|███▍      | 2112/6199 [1:37:44<2:48:05,  2.47s/it] 

Error in matchid: BR1_1921244640


 43%|████▎     | 2664/6199 [2:03:01<2:43:34,  2.78s/it]

...

 71%|███████   | 4399/6199 [3:20:21<1:14:27,  2.48s/it]

Error in matchid: BR1_1926509962


 74%|███████▍  | 4605/6199 [3:29:34<1:09:46,  2.63s/it]

Error in matchid: BR1_1924976674


100%|██████████| 6199/6199 [4:40:56<00:00,  2.72s/it]  


In [16]:
puuids_new.drop_duplicates()

,puuid
0,M38b7eC1_Qp2WSJgAi9F3Sd8N07mSMoNxml_TLcjBVRP5M...
1,wpKYtw4TqiWFEqkf5bTEb3GPb4ts3cO2Kqw2t4E_7-yOya...
2,Ql2lf25SCGPkTx6FB1mYNP0YkfjnIQYpT8y4fMPwnaYtAY...
3,XbTawWZCXlhyVxw-1GGk77wOIJHMnBx7yPl6fqAl02Yd73...
4,ioMFXi24_6KA-0VOmhnkEYPIpLKufqo3wKGpsEqSHNle88...
...,...
54472,w0qlMfBgnhgjxRUJGtlMvzIWbw_aB3FTinp0DbuzdLCTQI...
54473,QitA-RAz7MFGT0uAdViDpejn_TOSl4aE-FWpbjWlHAxte0...
54474,9xpTev6W9WZh8WjZPWEC93O6UN_yM41S1HPWsVEb2eFZnZ...
54484,giRV7UPwE009JNtBnIEbqZ9WH7BUaztmxwK-qCoaAYN3xX...


In [17]:
pd.read_csv('puuids.csv')

,puuid
0,M38b7eC1_Qp2WSJgAi9F3Sd8N07mSMoNxml_TLcjBVRP5M...
1,wpKYtw4TqiWFEqkf5bTEb3GPb4ts3cO2Kqw2t4E_7-yOya...
2,Ql2lf25SCGPkTx6FB1mYNP0YkfjnIQYpT8y4fMPwnaYtAY...
3,XbTawWZCXlhyVxw-1GGk77wOIJHMnBx7yPl6fqAl02Yd73...
4,ioMFXi24_6KA-0VOmhnkEYPIpLKufqo3wKGpsEqSHNle88...
...,...
18528,w0qlMfBgnhgjxRUJGtlMvzIWbw_aB3FTinp0DbuzdLCTQI...
18529,QitA-RAz7MFGT0uAdViDpejn_TOSl4aE-FWpbjWlHAxte0...
18530,9xpTev6W9WZh8WjZPWEC93O6UN_yM41S1HPWsVEb2eFZnZ...
18531,giRV7UPwE009JNtBnIEbqZ9WH7BUaztmxwK-qCoaAYN3xX...


# Adicionando novos matchids

In [18]:
puuids = pd.read_csv('puuids.csv')['puuid']
puuids

0        M38b7eC1_Qp2WSJgAi9F3Sd8N07mSMoNxml_TLcjBVRP5M...
1        wpKYtw4TqiWFEqkf5bTEb3GPb4ts3cO2Kqw2t4E_7-yOya...
2        Ql2lf25SCGPkTx6FB1mYNP0YkfjnIQYpT8y4fMPwnaYtAY...
3        XbTawWZCXlhyVxw-1GGk77wOIJHMnBx7yPl6fqAl02Yd73...
4        ioMFXi24_6KA-0VOmhnkEYPIpLKufqo3wKGpsEqSHNle88...
                               ...                        
18528    w0qlMfBgnhgjxRUJGtlMvzIWbw_aB3FTinp0DbuzdLCTQI...
18529    QitA-RAz7MFGT0uAdViDpejn_TOSl4aE-FWpbjWlHAxte0...
18530    9xpTev6W9WZh8WjZPWEC93O6UN_yM41S1HPWsVEb2eFZnZ...
18531    giRV7UPwE009JNtBnIEbqZ9WH7BUaztmxwK-qCoaAYN3xX...
18532    yj1cIjidZWqc3FKoNctp3MBHVWzujoMJiQFvY_yatwsDUD...
Name: puuid, Length: 18533, dtype: object

In [19]:
matchids_new = pd.read_csv('matchids.csv')
matchids_new

,matchid
0,BR1_1929504335
1,BR1_1917107281
2,BR1_1925813673
3,BR1_1929034883
4,BR1_1927766146
...,...
6194,BR1_1933014711
6195,BR1_1932950938
6196,BR1_1932935031
6197,BR1_1946164876


In [20]:
for i in tqdm(puuids):
    try:
        intermediate = riot.match_list(key, i)
        time.sleep(1)
        for j in intermediate:
            matchids_new = matchids_new.append({'matchid' : j}, ignore_index = True)
    except:
        print('Error in puuid:', i)
    
    matchids_new.drop_duplicates().to_csv('matchids.csv', index = False)

100%|██████████| 18533/18533 [15:38:58<00:00,  3.04s/it]  


In [22]:
matchids_new.drop_duplicates()

,matchid
0,BR1_1929504335
1,BR1_1917107281
2,BR1_1925813673
3,BR1_1929034883
4,BR1_1927766146
...,...
150496,BR1_1905789613
150497,BR1_1905624086
150498,BR1_1904733121
150499,BR1_1904627118


# Adicionando novas match_info

In [ ]:
matchids = pd.read_csv('matchids.csv')

In [ ]:
match_info = pd.read_csv('match_info.csv')
match_info

In [ ]:
for i in tqdm(matchids['matchid']):
    try:
        intermediate = riot.match_info(key, i)
        match_info = match_info.append(intermediate, ignore_index = True)
        match_info.drop_duplicates().to_csv('match_info.csv', index = False)
    except:
        print('Error in matchid:', i)
    
    time.sleep(0.5)

In [ ]:
match_info